# Setup

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    tpu_strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", tpu_strategy.num_replicas_in_sync)
import cv2
import numpy as np
from matplotlib import pyplot as plt
from skimage.io import imread
import math
from sklearn import model_selection as ms
from numpy import random
random.seed(0)

# In[2]: Define how to read in images
def read_images(arr, index, filepath, filepath2):
    datafiles = os.listdir(filepath)
    for f in datafiles:
        img = imread(os.path.join(filepath, f))
        arr[index,:,:,:] = img
        index += 1
    datafiles = os.listdir(filepath, filepath2)
    for f in datafiles:
        img = imread(os.path.join(filepath2,f))
        pts1 = np.float32([[0, 0],[224, 0],[0, 224],[224, 224]]) 
        pts2 = np.float32([[-10, 0],[200, 20],[20, 200],[220,235]])
        aug_img = cv2.warpPerspective(img, cv2.getPerspectiveTransform(pts1, pts2), (224,224))
        arr[index,:,:,:] = cv2.rotate(aug_img,cv2.ROTATE_90_CLOCKWISE)
        index+=1
        arr[index,:,:,:] = cv2.rotate(aug_img,cv2.ROTATE_90_COUNTERCLOCKWISE)
        index+=1
        arr[index,:,:,:] = cv2.convertScaleAbs(cv2.flip(img,1), 1, -8)
        index+=1
        arr[index,:,:,:] = cv2.convertScaleAbs(cv2.flip(img,0), 1, -8)
        index+=1
        arr[index,:,:,:] = cv2.convertScaleAbs(cv2.flip(img,1), 1, 8)
        index+=1
        arr[index,:,:,:] = cv2.convertScaleAbs(cv2.flip(img,0), 1, 8)
        index+=1
        arr[index,:,:,:] = cv2.flip(aug_img,1)
        index+=1
        arr[index,:,:,:] = cv2.flip(aug_img,0)
        index+=1


# In[3]: convert from integers to floats
def prep_pixels(train,test):
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # normalize to range 0-1
    train_norm /= 255.0
    test_norm /= 255.0
    # return normalized images
    return train_norm, test_norm


# In[7]: set up path to directories
mpdatapath='/kaggle/input/monkeypox-images/archive/Augmented Images/Monkeypox_augmented'
mporigpath='/kaggle/input/monkeypox-images/archive/Original Images/Monkey Pox'
otherdatapath='/kaggle/input/monkeypox-images/archive/Augmented Images/Others_augmented'
otherorigpath='/kaggle/input/monkeypox-images/archive/Original Images/Others'

# In[8]: Read in images and set up labels
M = len(os.listdir(mpdatapath)) + 8*len(os.listdir(mporigpath))
N = len(os.listdir(otherdatapath)) + 8*len(os.listdir(otherorigpath))
Xt = np.zeros((M+N,224,224,3))
yt = np.zeros(M+N)
yt[:M]=1

read_images(Xt, 0, mpdatapath, mporigpath)
print("number of mp files: ", M)
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.imshow(Xt[i]/255.0)
plt.show()

read_images(Xt, M, otherdatapath, otherorigpath)
print("number of other files: ", Xt.shape[0]-M)
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.imshow(Xt[M+i]/255.0)
plt.show()
yt.reshape((-1,1))


X, X_test, y, y_test = ms.train_test_split(Xt, yt, test_size=0.3, random_state=10)
# Clear up some memory
del Xt, yt, N, M
import gc
gc.collect()

print("X size: ", len(X))
print("test size: ", len(X_test))

In [ ]:
X,X_test = prep_pixels(X,X_test)

# Train Model

In [ ]:
def train_plot(tr_data, start_epoch):
    #Plot the training and validation data
    tacc=tr_data.history['binary_accuracy']
    tloss=tr_data.history['loss']
    vacc=tr_data.history['val_binary_accuracy']
    vloss=tr_data.history['val_loss']
    Epoch_count=len(tacc)+ start_epoch
    Epochs=[]
    for i in range (start_epoch ,Epoch_count):
        Epochs.append(i+1)   
    index_loss=np.argmin(vloss)#  this is the epoch with the lowest validation loss
    val_lowest=vloss[index_loss]
    index_acc=np.argmax(vacc)
    acc_highest=vacc[index_acc]
    plt.style.use('fivethirtyeight')
    sc_label='best epoch= '+ str(index_loss+1 +start_epoch)
    vc_label='best epoch= '+ str(index_acc + 1+ start_epoch)
    fig,axes=plt.subplots(nrows=1, ncols=2, figsize=(20,8))
    axes[0].plot(Epochs,tloss, 'r', label='Training loss')
    axes[0].plot(Epochs,vloss,'g',label='Validation loss' )
    axes[0].scatter(index_loss+1 +start_epoch,val_lowest, s=150, c= 'blue', label=sc_label)
    axes[0].set_title('Training and Validation Loss')
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[1].plot (Epochs,tacc,'r',label= 'Training Accuracy')
    axes[1].plot (Epochs,vacc,'g',label= 'Validation Accuracy')
    axes[1].scatter(index_acc+1 +start_epoch,acc_highest, s=150, c= 'blue', label=vc_label)
    axes[1].set_title('Training and Validation Accuracy')
    axes[1].set_xlabel('Epochs')
    axes[1].set_ylabel('Accuracy')
    axes[1].legend()
    plt.tight_layout    
    plt.show()

In [ ]:
from tensorflow import keras
from tensorflow.keras import utils as np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.regularizers import l2
import pickle

In [ ]:
def run_model(X_train, y_train, X_val, y_val, X_test, y_test):
    keras.backend.clear_session()
    with tpu_strategy.scope():
        model = Sequential([
            Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(224,224, 3)),
            MaxPooling2D((2, 2)),
            Dropout(0.12),
            Conv2D(32, (3, 3), padding='same', activation='relu'),
            MaxPooling2D((2, 2)),
            Dropout(0.12),
            Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(l=0.001)),
            MaxPooling2D((2, 2)),
            Dropout(0.12),
            Flatten(),
            Dense(128, activation='relu', kernel_regularizer=l2(l=0.007)),
            Dropout(0.5),
            Dense(1, activation='sigmoid'),
        ])
        opt = keras.optimizers.Adam(epsilon=0.01)
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['binary_accuracy'])
    
    early_stopping = keras.callbacks.EarlyStopping(
        monitor='val_binary_accuracy',
        patience=25,
        min_delta=0.001,
        restore_best_weights=True,
    )
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=120, batch_size=20, callbacks=[early_stopping])
    train_plot(history,0)
    score = model.evaluate(X_test, y_test)
    print('score: ', score[1])
    predY =  model.predict(X_test)
    predY = (predY > 0.5)
    y_pred = np.zeros(len(predY))
    for i in range(len(predY)):
        y_pred[i] = predY[i][0]
    cm = confusion_matrix(y_test, y_pred)
    class_names=['Monkeypox','Other']
    plot_confusion_matrix(cm, colorbar=True, show_normed=True, class_names=class_names)
    plt.show()
    model.save_weights(f'/kaggle/working/model_{str(int(score[1]*10000))}.hdf5')
    pickle.dump(model, open(f'model_{str(int(score[1]*10000))}.pkl', 'wb'))
    return score[1], cm

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

In [ ]:
SPLITS=5

In [ ]:
kf = StratifiedKFold(n_splits=SPLITS, shuffle=True, random_state=1)
results = np.zeros(SPLITS)
i = 0
cm_array = np.zeros((SPLITS,2,2))
for train_index, val_index in kf.split(X,y):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    results[i], cm_array[i,:,:] = run_model(X_train, y_train, X_val, y_val, X_test, y_test)
    i+=1

In [ ]:
np.average(results)

In [ ]:
print(results)

In [ ]:
print(cm_array)

In [ ]:
final_cm = np.zeros((2,2))
for i in range(5):
    for j in range(2):
        for k in range(2):
            final_cm[j][k] += cm_array[i,j,k]
class_names = ['Monkeypox', 'Other']
final_cm = final_cm.astype(int)
fig, ax = plot_confusion_matrix(final_cm, colorbar=True, show_normed=True, class_names=class_names)
plt.show()